## Non-parallel Game of Life

In [704]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [705]:
def Rand_gen(pos, ratio = 0.1):
    #ratio = 0.06
    pos = np.random.choice([0, 1], size=(pos.shape[0], pos.shape[1]), p=[1 - ratio, ratio])
    return pos

def Glider(pos):
    j = -1
    pos[1+j, 2+j] = pos[2+j, 3+j] = pos[3+j, 1+j] = pos[3+j, 2+j] = pos[3+j, 3+j] = 1
    return pos

def Beacon(pos):
    j = 1
    pos[1+j, 1+j] = pos[1+j, 2+j] = pos[2+j, 1+j] = pos[2+j, 2+j] = 1
    pos[3+j, 3+j] = pos[3+j, 4+j] = pos[4+j, 3+j] = pos[4+j, 4+j] = 1
    return pos

def Puffer(pos):
    j = 10
    pos[3+j, 1+j] = pos[4+j, 1+j] = pos[5+j, 1+j] = pos[10+j, 1+j] = 1
    pos[2+j, 2+j] = pos[5+j, 2+j] = pos[9+j, 2+j] = pos[10+j, 2+j] = pos[11+j, 2+j] = 1
    pos[2+j, 2+j] = pos[5+j, 3+j] = pos[9+j, 3+j] = pos[11+j, 3+j] = pos[12+j, 3+j] = 1
    pos[16+j, 3+j] = pos[17+j, 3+j] = pos[18+j, 3+j] = 1
    pos[1+j, 4+j] = pos[5+j, 4+j] = pos[10+j, 4+j] = pos[11+j, 4+j] = pos[12+j, 4+j] = 1
    pos[10+j, 4+j] = pos[15+j, 4+j] = pos[18+j, 4+j] = 1
    pos[5+j, 5+j] = pos[10+j, 5+j] = pos[11+j, 5+j] = pos[18+j, 5+j] = 1
    pos[1+j, 6+j] = pos[4+j, 6+j] = pos[10+j, 6+j] = pos[14+j, 6+j] = pos[18+j, 6+j] =1
    
    pos[13+j, 7+j] = pos[18+j, 7+j] = pos[18+j, 8+j] = pos[15+j, 9+j] = pos[17+j, 9+j] = 1
    
    return pos

def Gosper(pos):
    j = 1
    pos[6+j, 2+j] = pos[6+j, 3+j] = pos[7+j, 2+j] = pos[7+j, 3+j]  = 1
    pos[4+j, 36+j] = pos[4+j, 37+j] = pos[5+j, 36+j] = pos[5+j, 37+j]  = 1
    
    pos[6+j, 12+j] = pos[7+j, 12+j] = pos[8+j, 12+j] = 1
    pos[5+j, 13+j] = pos[4+j, 14+j] = pos[9+j, 13+j] = pos[10+j, 14+j] = 1
    pos[4+j, 15+j] = pos[10+j, 15+j] = 1
    pos[7+j, 16+j] = pos[5+j, 17+j] = pos[9+j, 17+j] = 1
    pos[7+j, 18+j] = pos[6+j, 18+j] = pos[8+j, 18+j] = pos[7+j, 19+j] = 1
    
    pos[6+j, 22+j] = pos[5+j, 22+j] = pos[4+j, 22+j] = 1
    pos[6+j, 23+j] = pos[5+j, 23+j] = pos[4+j, 23+j] = 1
    pos[7+j, 24+j] = pos[3+j, 24+j] = 1
    pos[7+j, 26+j] = pos[3+j, 26+j] = pos[8+j, 26+j] = pos[2+j, 26+j] = 1
    
    
    return pos

In [706]:
def neighbors_alive(pos, i, j): # Pretty easy intuitive model for all 8 black neigbours
    n = 0
    
    if pos[(i - 1) % pos.shape[0], j] == 1: # left
        n+=1
    if pos[i, (j - 1) % pos.shape[1]] == 1: # down
        n+=1
    if pos[(i - 1) % pos.shape[0], (j - 1) % pos.shape[1]] == 1: # down-left
        n+=1
    if pos[(i - 1) % pos.shape[0], (j + 1) % pos.shape[1]] == 1: # up-left
        n+=1

    if pos[i, (j + 1) % pos.shape[1]] == 1: # up
        n+=1
    if pos[(i + 1) % pos.shape[0], j] == 1: # right
        n+=1
    if pos[(i + 1) % pos.shape[0], (j - 1) % pos.shape[1]] == 1: # down-right
        n+=1
    if pos[(i + 1) % pos.shape[0], (j + 1) % pos.shape[1]] == 1: # up-right
        n+=1
        
    return n


def unhappy_agents(pos):
    a_i_r = []
    a_j_r = []
    a_i_k = []
    a_j_k = []
    for i in range(pos.shape[0]):
        for j in range(pos.shape[1]):
            N_alive = neighbors_alive(pos, i, j)
            
            if pos[i, j] == 0:
                if N_alive == 3:
                    #print('Dead cell:' ,i, j, '; N_neigb =', N_alive, ' - Resurrect')
                    a_i_r.append(i)
                    a_j_r.append(j)
            else:
                if N_alive < 2 or N_alive > 3 :
                    #print('Alive cell:' ,i, j, '; N_neigb =', N_alive, ' - Kill')
                    a_i_k.append(i)
                    a_j_k.append(j)  
                
    return a_i_r, a_j_r, a_i_k, a_j_k

## Relevant version

In [710]:
N = 50
Range = 200 # How many rounds do we have
k = 0
ratio = 0.2

pos = np.zeros((N, N))

#pos = Rand_gen(pos, ratio)
#pos = Beacon(pos)
#pos = Glider(pos)
#pos = Gosper(pos)
pos = Puffer(pos)

pos_array = []
alive_history = []
pos_array.append(pos.copy())
    
for p in range(Range):
    alive_history.append(sum(pos[pos != 0]))
    
    array_i_r, array_j_r, array_i_k, array_j_k = unhappy_agents(pos)

    for i in range(len(array_i_r)):
        pos[array_i_r[i], array_j_r[i]] = 1
    for i in range(len(array_i_k)):
        pos[array_i_k[i], array_j_k[i]] = 0
    pos_array.append(pos.copy())

In [711]:
%matplotlib auto
fig = plt.figure(figsize = (14,11))
plt.rcParams.update({'font.size': 20})
im = plt.imshow(pos, vmin=0, vmax=1, cmap='gist_yarg')

def animate(i): # Animation 
    im.set_array(pos_array[i]) 

ani = animation.FuncAnimation(fig, animate, len(pos_array))


#name = 'Random initial condition'
name = 'Puffer'
#name = 'Gosper gun'
#name = 'Glider'
#name = 'Beacon'

#ani.save('Task8_' + name + '.gif', writer='PillowWriter')

Using matplotlib backend: Qt5Agg


In [712]:

plt.figure(figsize = (14,11))  
plt.plot(range(len(alive_history)), alive_history, 'x-', color = 'r')
plt.rcParams.update({'font.size': 20})
plt.title(name) 
plt.xlabel('N, iteration')
plt.ylabel('Number of alive cells')
plt.show()

#plt.savefig('Task8_' + name + '.png', format='png')#

https://www.conwaylife.com/wiki/Main_Page